In [2]:
!pip install surya-ocr marker-pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.5 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [3]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)


/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded layout model datalab-to/surya_layout on device cuda with dtype torch.float16
Loaded texify model datalab-to/texify on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded table recognition model datalab-to/surya_tablerec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


In [11]:
import os
import hashlib


def calculate_hash(file_path):
    """Вычисляет SHA-256 хеш файла."""
    sha256 = hashlib.sha256()
    with open(file_path, 'rb') as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

def process_pdf_folder(folder_path, output_folder):
    """Обрабатывает папку с PDF файлами, конвертирует их в текст и исключает дубликаты."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    seen_hashes = set()
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(folder_path, filename)
            file_hash = calculate_hash(file_path)

            if file_hash not in seen_hashes:
                seen_hashes.add(file_hash)
                text = pdf_to_text(file_path)
                output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")
                with open(output_file_path, 'w', encoding='utf-8') as output_file:
                    output_file.write(text)
                print(f"Processed: {filename}")
            else:
                print(f"Duplicate found, skipping: {filename}")

def pdf_to_text(fname):
    rendered = converter(fname)
    text, _, images = text_from_rendered(rendered)
    return text

In [ ]:
folder_path = 'pdf_files'
output_folder = 'txt_files3'
process_pdf_folder(folder_path, output_folder)

100%|██████████| 12/12 [00:00<00:00, 48.40it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 6/6 [00:03<00:00,  1.54it/s]


Processed: 1.pdf


100%|██████████| 2/2 [00:00<00:00, 76.59it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 2/2 [00:00<00:00,  3.41it/s]

Processed: 285 (1).pdf
Duplicate found, skipping: 285.pdf



100%|██████████| 36/36 [00:00<00:00, 52.81it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 100%|██████████| 7/7 [00:44<00:00,  6.37s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 7/7 [00:05<00:00,  1.28it/s]


...

Processed: tkachov_less_is_more_why_oberon_beats_mainstream_1.pdf


100%|██████████| 2/2 [00:00<00:00, 117.72it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]

Processed: tkachov_oberon-new_dimensions_for_design_of_algorithms_for_scientific_applications.pdf



Recognizing Text: 100%|██████████| 52/52 [00:59<00:00,  1.14s/it]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]


Processed: tkachov_sistema_obrazovaniya_kak_faktor_nacionalnogo_suvereniteta_v_sfere_it.pdf
